# Importing the Libraries

In [1]:
import time


import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import gensim
import sys
import string
import time
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error, make_scorer
import random
import warnings
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn import cross_validation
import multiprocessing
from nltk.corpus import stopwords
from SpellCheck import spell_check
import logging

C:\Users\ravi\Anaconda2\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.
C:\Users\ravi\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# A Function to convert a document to a sequence of words

In [4]:

def prodinfo_to_wordlist( prodinfo_text):
    # Function to convert a document to a sequence of words,
    # 1. Remove non-letters
    prodinfo_text = re.sub("[^a-zA-Z]"," ", prodinfo_text)
    # 2. Convert words to lower case, split them and remove stop words
    stops = set(stopwords.words("english"))
    words = " ".join([word for word in spell_check(prodinfo_text.lower()).split() if word not in stops])
    # 3. Return a list of words
    return(words)

# A function to split a prodinfo into parsed sentences

In [5]:
def prodinfo_to_sentences( prodinfo, tokenizer):
    # Function to split a prodinfo into parsed sentences. Returns a
    # list of sentences, where each sentence is a list of words
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(prodinfo.strip())
    # raw_sentences = tokenizer.tokenize(prodinfo.decode('utf-8').strip())
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
    # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call prodinfo_to_wordlist to get a list of words
            sentences.append( prodinfo_to_wordlist( raw_sentence ))
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences


# Function to average all of the word vectors in a given paragraph

In [6]:
def makeFeatureVec(words, model, num_features):
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    # Index2word is a list that contains the names of the words in
    # the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    featureVec = np.divide(featureVec,nwords)
    return featureVec

# Given a set of prodinfos (each one a list of words), calculate
# the average feature vector for each one and return a 2D numpy array

In [7]:
def getAvgFeatureVecs(prodinfos, model, num_features):
    # Initialize a counter
    counter = 0.
    # Preallocate a 2D numpy array, for speed
    prodinfoFeatureVecs = np.zeros((len(prodinfos),num_features),dtype="float32")
    # Loop through the prodinfos
    for prodinfo in prodinfos:
       #
       # Print a status message every 1000th prodinfo
       if counter%100. == 0.:
           print "done %d of %d" % (counter, len(prodinfos))
       # Call the function (defined above) that makes average feature vectors
       prodinfoFeatureVecs[counter] = makeFeatureVec(prodinfo, model, num_features)
       #Increment the counter
       counter = counter + 1.
    return prodinfoFeatureVecs

In [8]:
def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

RMSE = make_scorer(fmean_squared_error, greater_is_better=False)

def RMSE(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

In [9]:
def run_est_func(params):
    n_estimators, learning_rate, max_depth, subsample = params
    n_estimators=int(n_estimators)
    print params
    clf = GradientBoostingRegressor(n_estimators= n_estimators, learning_rate=learning_rate, max_depth=max_depth, subsample=subsample)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    rmse = RMSE( y_test, y_pred )
    return rmse

In [10]:
def optimize(trials):
    space = (
             hp.quniform('n_estimators', 100,200, 100),
             hp.quniform('learning_rate', 0.1, 0.2, 0.1),
             hp.quniform('max_depth', 5, 6, 1),
             hp.quniform('subsample', 0.5, 1, 0.5)
             )
    best = fmin(run_est_func, space, algo=tpe.suggest, trials=trials, max_evals=10)
    print best

random.seed(2016)
stemmer = SnowballStemmer('english')

In [11]:
df_train = pd.read_csv('Data/train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('Data/test.csv', encoding="ISO-8859-1")
df_pro_desc = pd.read_csv('Data/product_descriptions.csv',encoding="ISO-8859-1")
df_attribute = pd.read_csv('Data/attributes.csv',encoding="ISO-8859-1")

# Looking at the data set

In [12]:
df_attribute.head(5)

,product_uid,name,value
0,100001.0,Bullet01,Versatile connector for various 90Â° connectio...
1,100001.0,Bullet02,Stronger than angled nailing or screw fastenin...
2,100001.0,Bullet03,Help ensure joints are consistently straight a...
3,100001.0,Bullet04,Dimensions: 3 in. x 3 in. x 1-1/2 in.
4,100001.0,Bullet05,Made from 12-Gauge steel


In [13]:
df_attribute[df_attribute.name == "Material"][["product_uid", "value"]].head()

,product_uid,value
8,100001.0,Galvanized Steel
67,100003.0,Composite
202,100007.0,Plastic
255,100009.0,Medium Density Fiberboard (MDF)
282,100010.0,Steel


In [14]:
df_train = pd.read_csv('Data/train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('Data/test.csv', encoding="ISO-8859-1")
df_pro_desc = pd.read_csv('Data/product_descriptions.csv',encoding="ISO-8859-1")
df_attribute = pd.read_csv('Data/attributes.csv',encoding="ISO-8859-1")
df_brand = df_attribute[df_attribute.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
df_material = df_attribute[df_attribute.name == "Material"][["product_uid", "value"]].rename(columns={"value": "material"})

df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid')
df_material1 = df_material.drop_duplicates(['product_uid'])
df_all = pd.merge(df_all, df_material1, how='left', on='product_uid')
print df_all.shape

(240760, 8)


In [15]:
df_all.head()

,id,product_title,product_uid,relevance,search_term,product_description,brand,material
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie,Galvanized Steel
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie,Galvanized Steel
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,BEHR Premium Textured DECKOVER is an innovativ...,BEHR Premium Textured DeckOver,NaN
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,Update your bathroom with the Delta Vero Singl...,Delta,NaN
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet,Update your bathroom with the Delta Vero Singl...,Delta,NaN


In [16]:
reload (sys)
sys.setdefaultencoding('ISO-8859-1')

stop_w = ['for', 'xbi', 'and', 'in', 'th','on','sku','with','what','from','that','less','er','ing'] #'electr','paint','pipe','light','kitchen','wood','outdoor','door','bathroom'
strNum = {'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9}

print ("String clean ups...")
 
df_all['brand'] = np.where(df_all['brand'].isin (['n a ','na',' na','nan']),'unbrand',df_all['brand'])

regex = re.compile('[%s]' % re.escape(string.punctuation))
df_all["search_term"] = df_all["search_term"].map(lambda x: regex.sub("", x))
df_all["product_title"] = df_all["product_title"].map(lambda x: regex.sub("", x))
df_all["product_description"] = df_all["product_description"].map(lambda x: regex.sub("", x))
df_all['combined_info'] = df_all['search_term'] + " " + \
                                    df_all['product_title'] + " " + \
                                    df_all['product_description']

df_all.to_csv('Data/df_all_vect.csv',index=False)


In [17]:
df_all['combined_info'] = df_all['search_term'] + " " + \
                                    df_all['product_title'] + " " + \
                                    df_all['product_description']

df_all.to_csv('Data/df_all_vect.csv',index=False)

# Combined Feature 

In [18]:
df_all.combined_info.head()

0    angle bracket Simpson StrongTie 12Gauge Angle ...
1    l bracket Simpson StrongTie 12Gauge Angle Not ...
2    deck over BEHR Premium Textured DeckOver 1gal ...
3    rain shower head Delta Vero 1Handle Shower Onl...
4    shower only faucet Delta Vero 1Handle Shower O...
Name: combined_info, dtype: object

# We have everything now lets start training the model

In [19]:
df_all["combined_info"].size

240760

# Load the punkt tokenizer

In [20]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle') 

In [21]:
sentences = []  # Initialize an empty list of sentences
from gensim.models import Word2Vec
print "Parsing sentences from training set"
for prodinfo in df_all["combined_info"]:
    sentences += prodinfo_to_sentences(prodinfo, tokenizer)

In [23]:
token_count = sum([len(sentence) for sentence in sentences])
print("The book corpus contains {0:,} tokens".format(token_count))

In [24]:
token_count

183613153

# Import the built-in logging module and configure it so that Word2Vec
# creates nice output messages

In [22]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', \
                    level=logging.INFO)

# Set values for various parameters

In [26]:
#ONCE we have vectors
#step 3 - build model
#3 main tasks that vectors help with
#DISTANCE, SIMILARITY, RANKING

# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
size = 300
# Minimum word count threshold.
min_count = 3
# Number of threads to run in parallel.
#more workers, faster we train
workers = multiprocessing.cpu_count()
# Context window length.
window = 20
# Downsample setting for frequent words.
#0 - 1e-5 is good for this
sample=1e-3
# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

# Initialize and train the model (this will take some time) 
uncomment line below to train the model again

In [27]:
print "Training Word2Vec model..."
import gensim.models.word2vec as w2v
model = w2v.Word2Vec(size = size, sg = 1, min_count = min_count, workers = workers, window = window
             , sample = sample, seed = seed, iter=10)

In [28]:
model.build_vocab(sentences)

In [29]:
len(model.wv.vocab)

27

In [ ]:
model.train(sentences, total_examples = model.corpus_count, epochs = model.iter)

In [85]:
model_name1 = "300features_30minwords_20context"
model.save(model_name)

# Loading the W2v model

In [120]:
model = gensim.models.Word2Vec.load("300features_30minwords_20context1")
print 'Model loaded'

# Compress the word vectors into 2D space and plot them

In [121]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)

In [122]:
all_word_vectors_matrix = thrones2vc.wv.syn0

# Training Tsne model, can take a while depending upon the machine its run

In [123]:
all_word_vectors_matrix_2d = tsne.fit_transform(all_word_vectors_matrix)

# Plotting all the words

In [124]:
points = pd.DataFrame(
    [
        (word, coords[0], coords[1])
        for word, coords in [
            (word, all_word_vectors_matrix_2d[thrones2vc.wv.vocab[word].index])
            for word in thrones2vc.wv.vocab
        ]
    ],
    columns=["word", "x", "y"]
)

In [125]:
import seaborn as sns
sns.set_context("poster")

points.plot.scatter("x", "y", s=10, figsize=(20, 12))

# Zoom in to some interesting places

In [ ]:
def plot_region(x_bounds, y_bounds):
    slice = points[
        (x_bounds[0] <= points.x) &
        (points.x <= x_bounds[1]) & 
        (y_bounds[0] <= points.y) &
        (points.y <= y_bounds[1])
    ]
    
    ax = slice.plot.scatter("x", "y", s=35, figsize=(10, 8))
    for i, point in slice.iterrows():
        ax.text(point.x + 0.005, point.y + 0.005, point.word, fontsize=11)

# Calculate average feature vectors

In [89]:
num_style_desc = df_all.shape[0]
for col in ['search_term','product_title','product_description']:
    cleantxt=[]
    for i in xrange( 0, num_style_desc ):
        if( (i+1)%10000== 0 ):
            print "vectors %d of %d\n" % ( i+1, num_style_desc )
        cleantxt.append(prodinfo_to_wordlist(df_all[col][i]) )
    print col
    vec = col+'_Vecs'
    num_features=300
    vec = getAvgFeatureVecs( cleantxt, model, num_features )
    print vec.shape
    df_vec = pd.DataFrame(vec,columns = ['vec_'+col+'_'+str(k) for k in range(num_features)])
    df_vec.to_csv('Data/'+col+'_word2vecs.csv',index=False)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
df_all = pd.read_csv('Data/df_all_vect.csv', encoding="ISO-8859-1")
srch_vec = pd.read_csv('Data/search_term_word2vecs.csv').fillna(0.0)
pdt_ttl_vec = pd.read_csv('Data/product_title_word2vecs.csv').fillna(0.0)
pdt_desc_vec = pd.read_csv('Data/product_description_word2vecs.csv').fillna(0.0)
srch_vec = srch_vec.as_matrix(columns=[srch_vec.columns[:300]])
pdt_ttl_vec = pdt_ttl_vec.as_matrix(columns=[pdt_ttl_vec.columns[:300]])
pdt_desc_vec = pdt_desc_vec.as_matrix(columns=[pdt_desc_vec.columns[:300]])

In [ ]:
dst_srch_ttl1 = np.zeros(srch_vec.shape[0])
for i in range(srch_vec.shape[0]):
    d1 = srch_vec[i, :]
    d2 = pdt_ttl_vec[i, :]
    dst_srch_ttl1[i] = cosine_similarity(d1, d2)
dst_srch_desc1 = np.zeros(srch_vec.shape[0])
for i in range(srch_vec.shape[0]):
    d1 = srch_vec[i, :]
    d2 = pdt_desc_vec[i, :]
    dst_srch_desc1[i] = cosine_similarity(d1, d2)
dst_ttl_desc1 = np.zeros(srch_vec.shape[0])
for i in range(srch_vec.shape[0]):
    d1 = pdt_ttl_vec[i, :]
    d2 = pdt_desc_vec[i, :]
    dst_srch_desc1[i] = cosine_similarity(d1, d2)

In [ ]:
svd = TruncatedSVD(n_components=30, random_state=2016)

In [ ]:
srch_vec = svd.fit_transform(srch_vec)
pdt_ttl_vec = svd.fit_transform(pdt_ttl_vec)
pdt_desc_vec = svd.fit_transform(pdt_desc_vec)

In [ ]:
srch_vec = pd.DataFrame(srch_vec, columns=['srch_vec_' + str(i) for i in range(srch_vec.shape[1])])
pdt_ttl_vec = pd.DataFrame(pdt_ttl_vec, columns=['ttl_vec_' + str(i) for i in range(pdt_ttl_vec.shape[1])])
pdt_desc_vec = pd.DataFrame(pdt_desc_vec, columns=['desc_vec_' + str(i) for i in range(pdt_desc_vec.shape[1])])

In [ ]:
id = list(df_all['id'])
srch_vec['id'] = id
pdt_ttl_vec['id'] = id
pdt_desc_vec['id'] = id

df_all = pd.merge(df_all, srch_vec, how='left', on='id')
df_all = pd.merge(df_all, pdt_ttl_vec, how='left', on='id')
df_all = pd.merge(df_all, pdt_desc_vec, how='left', on='id')

df_all['dst_srch_ttl1'] = dst_srch_ttl1
df_all['dst_srch_desc1'] = dst_srch_desc1
df_all['dst_ttl_desc1'] = dst_ttl_desc1

cols = list(df_all.select_dtypes(include=['object']).columns)

df_all1 = df_all.drop(cols, 1)
df_all1.to_csv('../../Data/df_all_new_feat3.csv', index=False)

In [ ]:
# Training
df_all = pd.read_csv('../../Data/df_all_new_feat3.csv', encoding="ISO-8859-1")

df_val = df_all[df_all['relevance'].isnull()]
df_train = df_all[~df_all['relevance'].isnull()]
id_val = df_val['id']
y_train = df_train['relevance'].values

X_train, X_test, y_train, y_test = train_test_split(df_train, y_train, test_size=0.3, random_state=1234)

X_train = X_train.drop(['id', 'relevance'], axis=1)
X_test = X_test.drop(['id', 'relevance'], axis=1)